In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
X, y = fetch_california_housing(data_home="./", return_X_y=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1,1)

In [4]:
class MyLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8,64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
 
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        y = self.fc3(x)
        return y

model = MyLinearRegression()

In [5]:
loss = nn.MSELoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 10000

In [8]:
# Datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
#Loaders
train_loader = DataLoader(train_dataset, batch_size=400, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=False)

In [ ]:
for epoch in range(epochs):
    for x_batch,y_batch in train_loader:
        yhat = model(x_batch)
        losses = loss(yhat, y_batch)
        losses.backward()
        optimizer.step()
        optimizer.zero_grad()
        if epoch % 10 == 0:
            print(f"epoch: {epoch}, loss: {losses.item()}")

In [9]:
for x_batch, y_batch in train_loader:
    print(x_batch.shape, y_batch.shape)

torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([400, 1])
torch.Size([400, 8]) torch.Size([4

In [ ]:
# model
class Regression(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
# mini-batch training
def mini_batch_train(model, train_loader, optimizer, loss_fn):
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        model.train()
        # step 1 前向计算预测值
        yhat = model(x_batch)
        # step 2 计算损失
        mini_batch_loss = loss_fn(yhat, y_batch)
        # step 3 反向传播计算梯度
        mini_batch_loss.backward()
        # step 4 更新参数
        optimizer.step()
        # 清空梯度
        optimizer.zero_grad()
        # 记录损失
        mini_batch_losses.append(mini_batch_loss.item())
    
    return np.mean(mini_batch_losses)

In [ ]:
# 测试
def mini_batch_val(model, test_loader, optimizer, loss_fn):
    mini_batch_losses = []
    for x_batch, y_batch in test_loader:
        # 置为验证状态
        model.eval()
        # Step 1 - 前向计算预测值
        yhat = model(x_batch)
        # Step 2 - 计算损失
        mini_batch_loss = loss_fn(yhat, y_batch)
        mini_batch_losses.append(mini_batch_loss.item())
    
    return np.mean(mini_batch_losses)

In [ ]:
# 训练
def train(model, train_loader, test_loader, optimizer, loss_fn, epochs):
    # 指定随机数种子，可以再现数据
    np.random.seed(23)
    torch.manual_seed(23)

    # 循环轮数计数
    total_epochs = 0
    # 记录训练和验证损失
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        # 训练
        model.train()
        total_epochs += 1
        train_loss = mini_batch_train(model, train_loader, optimizer, loss_fn)
        train_losses.append(train_loss)
        # 验证
        with torch.no_grad():
            val_loss = mini_batch_val(model, test_loader, optimizer, loss_fn)
            val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")
    
    return train_losses, val_losses

In [ ]:
model = Regression()
loss = nn.MSELoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 10
train_losses, val_losses = train(model, train_loader, test_loader, optimizer, loss, epochs)


In [ ]:
plt.style.use('fivethirtyeight')
def plot_losses():
    fig = plt.figure(figsize=(10, 4))
    plt.plot(train_losses, label='Training Loss', c='b')
    plt.plot(val_losses, label='Validation Loss', c='r')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    return fig

fig=plot_losses()

In [ ]:
test_data, test_target = next(iter(test_loader))

In [ ]:
test_data.shape, test_target.shape

In [ ]:
model.eval()
for i in range(16):
    yhat = model(test_data[i])
    print(f"Predicted value: {yhat.item():.4f}, Actual value: \
          {test_target[i].item():.4f}")
model.train()